In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_data = pd.read_csv("/kaggle/input/task-03/train_full.csv")
game = train_data.Game
y = train_data['WinOrLose']
train_data.drop(["Game", "WinOrLose"], axis = 1, inplace = True)

In [ ]:
train_data.shape

In [ ]:
train_data.head(10)

In [ ]:
test_data = pd.read_csv("/kaggle/input/task-03/test_without_label.csv")
game = test_data.Game
test_data.drop("Game", axis = 1, inplace = True)

In [ ]:
test_data.head()

### Feature Engineering

In [ ]:
test_data.columns

Os nomes das colunas apresentam um espaço em branco no final do nome, para facilitar vamos retirar esse espaço para as colunas do treino e teste:

In [ ]:
def transform_column_names (df):
    df.columns = [x.strip() for x in df.columns] 

In [ ]:
transform_column_names (train_data)
transform_column_names (test_data)

Função para transformar as datas em dia da semana, mês e dia do mês, talvez seja útil para uma análise posterior:

In [ ]:
def transform_dates(df):
    list_dates = []
    for date in df.Data:
        dates = [i for i in date.split(' ')]
        list_dates.append(dates)
    
    dates_df = pd.DataFrame(list_dates, columns=["week_day", "month", "day"])
    dates_df["week_day"] = dates_df["week_day"].str.strip(',')
    dates_df["day"] = dates_df["day"].astype(int)
    dates_df["month"] = dates_df["month"].apply(lambda x : datetime.datetime.strptime(x, "%B").month)
    df.drop("Data", axis=1, inplace=True)
    return dates_df

In [ ]:
dates_train = transform_dates(train_data)
dates_test = transform_dates(test_data)

In [ ]:
dates_train.month.value_counts()

In [ ]:
dates_test.month.value_counts()

Como podemos observar os playoffs geralmente ocorrem em abril, maio, junho (como visto no treino). Como podemos observar no teste na temporada 2019-2020, os playoffs ocorreram em agosto, setembro e outubro por conta da pandemia. Se quisermos usar a variável temporal devemos renomear essa datas não presentes no treino:

In [ ]:
replace_month = {8:4, 9:5, 10:6}
dates_test['month'] = dates_test['month'].replace(replace_month)

In [ ]:
dates_test.month.value_counts()

Como temos times desde a temporada 2006, procuramos quais foram os times que mudaram de nome nesse período:
- New Jersey Nets(1977–2012) e Brooklyn Nets desde 2012
- New Orleans/Oklahoma City Hornets(2005-2007), New Orleans Hornets(2007-2013), New Orleans Pelicans desde 2013
- Charlotte Hornets – Charlotte Hornets(1988–2002 e desde 2014) e Charlotte Bobcats(2004–2014)


Substituímos pelo nome mais recente:

In [ ]:
features = pd.concat([train_data, test_data]).reset_index(drop=True)

In [ ]:
def transform_team(df):
    dic_changed_teams = {"New Jersey Nets": "Brooklyn Nets",
           "New Orleans Hornets": "New Orleans Pelicans",
           "Charlotte Bobcats": "Charlotte Hornets"}
    df["H_Team"] = df["H_Team"].replace(dic_changed_teams)
    df["A_Team"] = df["A_Team"].replace(dic_changed_teams)
    return df

In [ ]:
transform_team(features)

Conferindo se as atualizações foram feitas corretamente nos times da casa e fora de casa (os conjuntos devem ser iguais):

In [ ]:
set(features.H_Team.unique())-set(features.A_Team.unique())

O ranking do time durante a temporada regular e os salários do time podem ajudar o modelo. É esperado que time com melhor colocação durante a temporada regular e com maior orçamento sejam favoritos em um jogo de play-offs:

In [ ]:
rank = pd.read_csv("/kaggle/input/dadosnba/rank.csv", sep = ";")
salario = pd.read_csv("/kaggle/input/dadosnba/salario.csv", sep = ";")

In [ ]:
replace_team_names = {'76ers':"Philadelphia 76ers", 'Nets':"Brooklyn Nets", 'Knicks':"New York Knicks",'Celtics':"Boston Celtics",
                      'Raptors': "Toronto Raptors",'Bucks':"Milwaukee Bucks", 'Pacers':"Indiana Pacers", 'Bulls':"Chicago Bulls", 
                      'Cavaliers': "Cleveland Cavaliers", 'Pistons': "Detroit Pistons",'Hawks': "Atlanta Hawks",
                      'Heat': "Miami Heat", 'Wizards': "Washington Wizards", 'Hornets': "Charlotte Hornets", 'Magic': "Orlando Magic",
                      'Jazz': "Utah Jazz", 'Nuggets': "Denver Nuggets", 'Blazers': "Portland Trail Blazers", 'Timberwolves': "Minnesota Timberwolves",
                      'Thunder': "Oklahoma City Thunder", 'Suns': "Phoenix Suns", 'Clippers': "Los Angeles Clippers" , 'Lakers':"Los Angeles Lakers",
                      'Warriors': "Golden State Warriors", 'Kings': "Sacramento Kings", 'Mavericks': "Dallas Mavericks",
                      'Grizzlies': "Memphis Grizzlies",'Spurs': "San Antonio Spurs", 'Pelicans':"New Orleans Pelicans", 'Rockets':"Houston Rockets"}

In [ ]:
salario['Time'] = salario['Time'].replace(replace_team_names)
salario.index = salario['Time']

In [ ]:
rank['Time'] = rank['Time'].replace(replace_team_names)
rank.index = rank['Time']

Criando uma variável para diferenciar o ano de cada partida:

In [ ]:
def create_year(df):
    examples_per_year = [89, 79, 86, 85, 82, 81, 84, 85, 89, 81, 86, 79, 82, 83]
    df["Year"] = np.repeat(list(range(2006, 2020)), examples_per_year)

In [ ]:
create_year(features)

Criando a variável ranking do time na temporada regular:

In [ ]:
def create_rank (df):
    list_h_rank = []
    for h_team, year in zip(df.H_Team, df.Year):
        list_h_rank.append(rank.loc[h_team,[str(year)]].values[0])
    list_a_rank = []
    for a_team, year in zip(df.A_Team, df.Year):
        list_a_rank.append(rank.loc[a_team,[str(year)]].values[0])
    df["H_rank"] = list_h_rank
    df["A_rank"] = list_a_rank

Criando a variável salary (orçamento dos salários do time durante a temporada):

In [ ]:
def create_salary (df):
    list_h_salary = []
    for h_team, year in zip(df.H_Team, df.Year):
        list_h_salary.append(salario.loc[h_team,[str(year)]].values[0])
    list_a_salary = []
    for a_team, year in zip(df.A_Team, df.Year):
        list_a_salary.append(salario.loc[a_team,[str(year)]].values[0])
    df["H_salary"] = list_h_salary
    df["A_salary"] = list_a_salary

In [ ]:
create_rank(features)

In [ ]:
create_salary(features)

Nosso feature engineering consiste em eliminar variáveis que representem a mesma informação (por exemplo existem variáveis que fazem parte do cálculo da porcentagem de ua estatística) e não é necessário mantê-las. Ao criar ``dif_features`` de tal modo que a estatatística do time da casa seja subtraída da estatística referente ao time visitante, também conseguimos manter a informação dos dados e eliminamos variáveis. Isso foi feito pois é esperado que se a diferença entre as variáveis do time da casa e vistiante for maior que 0 ou menor que 0, ela represente um maior indício de vitória seja para o time da casa ou visitante. Por exemplo se o time da casa tem uma maior porcentagem de acertos nas cestas de 3 comparado com o time visitante, é provável que ele faça mais pontos e ganhe a partida, e vice-versa. A mesma lógica foi aplicada para as outras variáveis:

In [ ]:
def create_dif_features (df):
    df['dif_rank'] = df['H_rank']-df['A_rank']
    df.drop(['H_rank', 'A_rank'], axis = 1, inplace = True)
    df['dif_salary'] = df['H_salary']-df['A_salary']
    df.drop(['H_salary', 'A_salary'], axis = 1, inplace = True)
    df['dif_W/D %'] = df['H_W/D %']-df['A_W/D %']
    df.drop(['H_Wins','H_Loss','H_W/D %', 'A_Wins','A_Loss','A_W/D %'], axis = 1, inplace = True)
    df['dif_FG%'] = df['H_FG%']-df['A_FG%']
    df.drop(['H_FG','H_FGA','H_FG%','A_FG','A_FGA','A_FG%'], axis = 1, inplace = True)
    df['dif_SRS'] = df['H_SRS']-df['A_SRS']
    df.drop(['H_SRS', 'A_SRS'], axis = 1, inplace = True)
    df['dif_AvgPointsPerGame'] = df['H_AvgPointsPerGame']-df['A_AvgPointsPerGame']
    df.drop(['H_Games','H_TotalPoints','H_AvgPointsPerGame', 'A_Games','A_TotalPoints','A_AvgPointsPerGame'], axis = 1, inplace = True)
    df['dif_3P%'] = df['H_3P%']-df['A_3P%']
    df.drop(['H_3P','H_3PA','H_3P%','A_3P','A_3PA','A_3P%'], axis = 1, inplace = True)
    df['dif_2P%'] = df['H_2P%']-df['A_2P%']
    df.drop(['H_2P','H_2PA','H_2P%','A_2P','A_2PA','A_2P%'], axis = 1, inplace = True)
    df['dif_FT%'] = df['H_FT%']-df['A_FT%']
    df.drop(['H_FT','H_FTA','H_FT%','A_FT','A_FTA','A_FT%'], axis = 1, inplace = True)
    df['dif_TRB'] = df['H_TRB']-df['A_TRB']
    df.drop(['H_ORB','H_DRB','H_TRB','A_ORB','A_DRB','A_TRB'], axis = 1, inplace = True)
    df['dif_AST'] = df['H_AST']-df['A_AST']
    df.drop(['H_AST','A_AST'], axis = 1, inplace = True)
    df['dif_BLK'] = df['H_BLK']-df['A_BLK']
    df.drop(['H_BLK','A_BLK'], axis = 1, inplace = True)
    df['dif_TOV'] = df['H_TOV']-df['A_TOV']
    df.drop(['H_TOV','A_TOV'], axis = 1, inplace = True)
    df['dif_PF'] = df['H_PF']-df['A_PF']
    df.drop(['H_PF','A_PF'], axis = 1, inplace = True)
    
    df['dif_AvgPointsPerGameOpp'] = df['H_AvgPointsPerGameOpp']-df['A_AvgPointsPerGameOpp']
    df.drop(['H_PointsOpp','H_AvgPointsPerGameOpp','A_PointsOpp','A_AvgPointsPerGameOpp'], axis = 1, inplace = True)
    df['dif_OFG%'] = df['H_OFG%']-df['A_OFG%']
    df.drop(['H_OFG','H_OFGA','H_OFG%','A_OFG','A_OFGA','A_OFG%'], axis = 1, inplace = True)
    df['dif_O3P%'] = df['H_O3P%']-df['A_O3P%']
    df.drop(['H_O3P','H_O3PA','H_O3P%','A_O3P','A_O3PA','A_O3P%'], axis = 1, inplace = True)
    df['dif_O2P%'] = df['H_O2P%']-df['A_O2P%']
    df.drop(['H_O2P','H_O2PA','H_O2P%','A_O2P','A_O2PA','A_O2P%'], axis = 1, inplace = True)
    df['dif_OFT%'] = df['H_OFT%']-df['A_OFT%']
    df.drop(['H_OFT','H_OFTA','H_OFT%','A_OFT','A_OFTA','A_OFT%'], axis = 1, inplace = True)
    df['dif_OTRB'] = df['H_OTRB']-df['A_OTRB']
    df.drop(['H_OORB','H_ODRB','H_OTRB','A_OORB','A_ODRB','A_OTRB'], axis = 1, inplace = True)
    df['dif_OAST'] = df['H_OAST']-df['A_OAST']
    df.drop(['H_OAST','A_OAST'], axis = 1, inplace = True)
    df['dif_OBLK'] = df['H_OBLK']-df['A_OBLK']
    df.drop(['H_OBLK','A_OBLK'], axis = 1, inplace = True)
    df['dif_OTOV'] = df['H_OTOV']-df['A_OTOV']
    df.drop(['H_OTOV','A_OTOV'], axis = 1, inplace = True)
    df['dif_OPF'] = df['H_OPF']-df['A_OPF']
    df.drop(['H_OPF','A_OPF'], axis = 1, inplace = True)
    
    df['dif_PW'] = df['H_PW']-df['A_PW']
    df.drop(['H_PW','A_PW'], axis = 1, inplace = True)
    df['dif_PL'] = df['H_PL']-df['A_PL']
    df.drop(['H_PL','A_PL'], axis = 1, inplace = True)
    df['dif_MOV'] = df['H_MOV']-df['A_MOV']
    df.drop(['H_MOV','A_MOV'], axis = 1, inplace = True)
    df['dif_SOS'] = df['H_SOS']-df['A_SOS']
    df.drop(['H_SOS','A_SOS'], axis = 1, inplace = True)
    df['dif_Ortg'] = df['H_Ortg']-df['A_Ortg']
    df.drop(['H_Ortg','A_Ortg'], axis = 1, inplace = True)
    df['dif_Drtg'] = df['H_Drtg']-df['A_Drtg']
    df.drop(['H_Drtg','A_Drtg'], axis = 1, inplace = True)
    df['dif_Pace'] = df['H_Pace']-df['A_Pace']
    df.drop(['H_Pace','A_Pace'], axis = 1, inplace = True)
    df['dif_Ftr'] = df['H_Ftr']-df['A_Ftr']
    df.drop(['H_Ftr','A_Ftr'], axis = 1, inplace = True)
    df['dif_3PAr'] = df['H_3PAr']-df['A_3PAr']
    df.drop(['H_3PAr','A_3PAr'], axis = 1, inplace = True)
    df['dif_TS%'] = df['H_TS%']-df['A_TS%']
    df.drop(['H_TS%','A_TS%'], axis = 1, inplace = True)
    df['dif_eFG%'] = df['H_eFG%']-df['A_eFG%']
    df.drop(['H_eFG%','A_eFG%'], axis = 1, inplace = True)
    df['dif_TOV%'] = df['H_TOV%']-df['A_TOV%']
    df.drop(['H_TOV%','A_TOV%'], axis = 1, inplace = True)
    df['dif_ORB%'] = df['H_ORB%']-df['A_ORB%']
    df.drop(['H_ORB%','A_ORB%'], axis = 1, inplace = True)
    df['dif_FT/FGA'] = df['H_FT/FGA']-df['A_FT/FGA']
    df.drop(['H_FT/FGA','A_FT/FGA'], axis = 1, inplace = True)
    df['dif_OeFG%'] = df['H_OeFG%']-df['A_OeFG%']
    df.drop(['H_OeFG%','A_OeFG%'], axis = 1, inplace = True)
    df['dif_OTOV%'] = df['H_OTOV%']-df['A_OTOV%']
    df.drop(['H_OTOV%','A_OTOV%'], axis = 1, inplace = True)
    df['dif_DRB%'] = df['H_DRB%']-df['A_DRB%']
    df.drop(['H_DRB%','A_DRB%'], axis = 1, inplace = True)
    df['dif_OFT/FGA'] = df['H_OFT/FGA']-df['A_OFT/FGA']
    df.drop(['H_OFT/FGA','A_OFT/FGA'], axis = 1, inplace = True)

In [ ]:
create_dif_features (features)

### EDA

In [ ]:
train_final = features.loc[0:train_data.shape[0]-1,:]
test_final = features.loc[train_data.shape[0]:,:]

In [ ]:
y_to_dict = {'L':0, 'W':1}
y_model_1 = y

In [ ]:
y_model_1 = y_model_1.map(y_to_dict)

In [ ]:
train_model_1 = pd.concat([train_final, y_model_1], axis =1)

Vamos ver a matriz de correlação entre as variávies númericas criadas:

In [ ]:
sns.set(font_scale=1.1)
correlation_train = train_model_1.corr()
mask = np.triu(correlation_train.corr())
plt.figure(figsize=(20, 20))
sns.heatmap(correlation_train,
            annot=True,
            fmt='.1f',
            cmap='coolwarm',
            square=True,
            mask=mask,
            linewidths=1,
            cbar=False)

plt.show()

Vamos pergar apenas a varáveis com correlação maior igual que 0.2:

In [ ]:
list(correlation_train['WinOrLose'][abs(correlation_train['WinOrLose'])>=0.2].index.values)[:-1]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import balanced_accuracy_score

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

new_train = train_final.select_dtypes(include=numerics)
new_test = test_final.select_dtypes(include=numerics)

Vamos fazer o boxplot das variáveis númericas com relação ao target:

In [ ]:
fig, axes = plt.subplots(nrows=11, ncols=4, figsize=(20, 17))
for idx, feat in enumerate(list(new_train.columns.values)[1:]):
    ax = axes[int(idx / 4), idx % 4]
    sns.boxplot(x=feat, y=y, data=new_train, ax=ax)
    ax.set_xlabel("")
    ax.set_ylabel(feat)
fig.tight_layout();

In [ ]:
train_model = train_final.loc[:,list(correlation_train['WinOrLose'][abs(correlation_train['WinOrLose'])>=0.2].index.values)[:-1]]
test_model = test_final.loc[:,list(correlation_train['WinOrLose'][abs(correlation_train['WinOrLose'])>=0.2].index.values)[:-1]]

In [ ]:
train_model.columns

É interessante observar que as variáveis com maior diferenças de distribuição entre vitórias (W) e derrotas (L) são justamente a maioria das variáveis com correlação maior igual que 0.2:

In [ ]:
train_model.loc[:,['dif_PL', 'dif_PW']].head(30)

A diferença entre vitórias pitorescas (PW) e derrotas pitorescas (PL) é a mesma em módulo. As variáveis representam a mesma informação, portanto não serão usadas no modelo:

In [ ]:
train_model.columns

Como podemos ver nosso modelo não usará as variáveis dif_ranking nem dif_salary. Além disso só estamos usando as variáveis númericas, visto que por exemplo os nomes dos times podem ser relevantes durante uma temporada, mas são mais importantes suas estatísticas de um ano para outro porque seus desempenhos mudam bastante

### Modelo

In [ ]:
X = train_model.drop(labels=['dif_PL'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y.map({'W': 1, 'L': 0}), test_size=0.2, random_state=17)

Testamos Random Forest, mas os resultado não foram bons, por isso nem tentamos algoritmos de boosting com árvores. Modelos como regressão logística, knn, naive bayes e svm foram usados:

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

**Gaussian Naive Bayes**

In [ ]:
from sklearn.model_selection import (GridSearchCV, StratifiedKFold,
                                     cross_val_score, cross_validate)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

nb = make_pipeline(StandardScaler(), GaussianNB())

results = cross_validate(nb, X_train, y_train, scoring= 'balanced_accuracy',cv=skf, verbose=True, return_train_score=True)

print("CV accuracy score: {:.2f}% +- {:.2f}%".format(results['train_score'].mean() * 100, results['train_score'].std() * 100))
nb.fit(X_train, y_train)
print("Train score:",balanced_accuracy_score(y_train, nb.predict(X_train)))
print("Holdout score:",balanced_accuracy_score(y_test, nb.predict(X_test)))

**Logistic Regression**

In [ ]:
lr = make_pipeline(StandardScaler(), LogisticRegression(random_state=17))
results = cross_validate(lr, X_train, y_train, scoring= 'balanced_accuracy',cv=skf, verbose=True, return_train_score=True)

print("CV accuracy score: {:.2f}% +- {:.2f}%".format(results['train_score'].mean() * 100, results['train_score'].std() * 100))
lr.fit(X_train, y_train)
print("Train score:",balanced_accuracy_score(y_train, lr.predict(X_train)))
print("Holdout score:",balanced_accuracy_score(y_test, lr.predict(X_test)))

**KNN**

In [ ]:
knn_pipe_ss = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1))])

knn_par_ss = {'knn__n_neighbors': range(3, 17), 
              'knn__weights':["uniform", "distance"], 
             'knn__metric':["euclidean", "minkowski", "manhattan"]}

knn_model = GridSearchCV(knn_pipe_ss, knn_par_ss, scoring= 'balanced_accuracy',cv=skf, verbose=True, return_train_score=True)
knn_model.fit(X_train, y_train)

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors=knn_model.best_params_['knn__n_neighbors'], metric = knn_model.best_params_["knn__metric"], weights=knn_model.best_params_["knn__weights"])
results = cross_validate(model_knn, X_train, y_train, scoring= 'balanced_accuracy',cv=skf, verbose=True, return_train_score=True)

print("CV accuracy score: {:.2f}% +- {:.2f}%".format(results['train_score'].mean() * 100, results['train_score'].std() * 100))
model_knn.fit(X_train, y_train)
print("Train score:",balanced_accuracy_score(y_train, model_knn.predict(X_train)))
print("Holdout score:",balanced_accuracy_score(y_test, model_knn.predict(X_test)))

**SVM**

In [ ]:
svm = make_pipeline(StandardScaler(), SVC(gamma=0.05, C=5,kernel='rbf'))
results = cross_validate(svm, X_train, y_train, scoring= 'balanced_accuracy',cv=skf, verbose=True, return_train_score=True)

print("CV accuracy score: {:.2f}% +- {:.2f}%".format(results['train_score'].mean() * 100, results['train_score'].std() * 100))
svm.fit(X_train, y_train)
print("Train score:",balanced_accuracy_score(y_train, svm.predict(X_train)))
print("Holdout score:",balanced_accuracy_score(y_test, svm.predict(X_test)))

### Submissão

O melhor modelo foi o Naive Bayes (treinamento e holdout não tiveram overfitting):

In [ ]:
print("All Train score:",balanced_accuracy_score(y.map({'W': 1, 'L': 0}), nb.predict(X)))

In [ ]:
test_id = pd.DataFrame(data = {"Game": game, "WinOrLose": pd.Series(nb.predict(test_model.drop(columns=['dif_PL']))).map({1:'W',0:'L'}).ravel()})
test_id.to_csv("resposta1.csv", index = False)
test_id.groupby('WinOrLose').count()

In [ ]:
pd.read_csv("resposta1.csv")